# Useful Notebook: Report Testing Data Prediction Probabilities and Illustrate Accessing Evaluation Metrics
**This notebook will (1) show users how to access all model evaluation metrics from internal pickle files, and (2) generate model (class 1) prediction probabilities for instances of the respective testing dataset.**

*This notebook is designed to run after having run STREAMLINE (at least phases 1-6) and will use the files from a specific STREAMLINE experiment folder, as well as save new output files to that same folder.*

***
## Notebook Details
STREAMLINE outputs pickled objects with (1) all the metric results, (2) elements needed to build the ROC and PRC plots, as well as (3) the prediction probabilities on the testing data across all datasets, algorithm models, and CV dataset partitions. 

This notebook illustrates how the user can access the pickled metric information saved as a list object. 

It includes (1) grabbing and calculating all average metric scores over the CV partitions, (2) grabbing the elements needed to build the average ROC plot, (3) grabbing the elementes needed to build the average PRC plot, (4) grabbing and reporting average model feature importance scores, and (5) grabbing and reporting the model testing prediction probabilities for each instance of the dataset. 

When run, this last item will generate a new folder (`prediction_probas`) in the pipeline's output experiment folder in the `model_evaluation` folder for each dataset. Here the class 1 prediction probabilities are reported as a `.csv` file for each algorithm and CV partition pair. In these files is the instance's true outcome value, the unique instance ID, and the predicted probability of the instance being class 1 (i.e. which typically encodes cases or the less frequent class). 
 

***
## Notebook Run Parameters
* This notbook has been set up to run 'as-is' on the experiment folder generated when running the demo of STREAMLINE in any mode (if no run parameters were changed). 
* If you have run STREAMLINE on different target data or saved the experiment to some other folder outside of STREAMLINE, you need to edit `experiment_path` below to point to the respective experiment folder.

In [1]:
experiment_path = "../DemoOutput/demo_experiment" # path the target experiment folder 
target_data_list = None # None if user wants to generate output for all analyzed target datasets, otherwise provide a (str) list of target dataset names to run
algorithms = [] # use empty list if user wishes re-evaluate all modeling algorithms that were run in pipeline, otherwise specify a (str) list of algorithm identifiers.

***
## Housekeeping
### Import Packages

In [2]:
import os
import pandas as pd
import pickle
import numpy as np
from statistics import mean
from scipy import interp,stats
import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Automatically Detect Dataset Names

In [3]:
# Get dataset paths for all completed dataset analyses in experiment folder
datasets = os.listdir(experiment_path)

# Name of experiment folder
experiment_name = experiment_path.split('/')[-1] 

datasets = os.listdir(experiment_path)
remove_list = ['.DS_Store', 'metadata.pickle', 'metadata.csv', 'algInfo.pickle',
                'DatasetComparisons', 'jobs', 'jobsCompleted', 'logs',
                'KeyFileCopy', 'dask_logs',
                experiment_name + '_STREAMLINE_Report.pdf']
for text in remove_list:
    if text in datasets:
        datasets.remove(text)

datasets = sorted(datasets) # ensures consistent ordering of datasets
print("Analyzed Datasets: " + str(datasets))

Analyzed Datasets: ['hcc_data', 'hcc_data_custom']


### Load Other Necessary Parameters

In [4]:
# Unpickle metadata from previous phase
file = open(experiment_path + '/' + "metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()
# Load variables specified earlier in the pipeline from metadata
outcome_label = metadata['Class Label']
instance_label = metadata['Instance Label']
cv_partitions = int(metadata['CV Partitions'])

# Unpickle algorithm information from previous phase
file = open(experiment_path + '/' + "algInfo.pickle", 'rb')
algInfo = pickle.load(file)
file.close()
algorithms = []
abbrev = {}
for key in algInfo:
    if algInfo[key][0]: # If that algorithm was used
        algorithms.append(key)
        abbrev[key] = (algInfo[key][1])

print("Algorithms Ran: " + str(algorithms))

Algorithms Ran: ['Decision Tree', 'Logistic Regression', 'Naive Bayes']


***
## From Pickle: Extract Metric List and Cacluate CV Averages

In [5]:
def print_results(algorithm, full_path):
    # Define evaluation stats variable lists
        s_bac = [] # balanced accuracies
        s_ac = [] # standard accuracies
        s_f1 = [] # F1 scores
        s_re = [] # recall values
        s_sp = [] # specificities
        s_pr = [] # precision values
        s_tp = [] # true positives
        s_tn = [] # true negatives
        s_fp = [] # false positives
        s_fn = [] # false negatives
        s_npv = [] # negative predictive values
        s_lrp = [] # likelihood ratio positive values
        s_lrm = [] # likelihood ratio negative values
        
        aucs = [] #areas under ROC curve
        praucs = [] #area under PRC curve
        aveprecs = [] #average precisions for PRC
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            metric_list = results[0] #First item in pickled list is the metric list (set of standard classification metrics)
            roc_auc = results[3] #Fourth item is the ROC AUC
            prec_rec_auc = results[6] #Seventh item is the PRC AUC
            ave_prec = results[7] #Eighth item is the average precision of PRC
            
            #Separate metrics from metricList
            s_bac.append(metric_list[0])
            s_ac.append(metric_list[1])
            s_f1.append(metric_list[2])
            s_re.append(metric_list[3])
            s_sp.append(metric_list[4])
            s_pr.append(metric_list[5])
            s_tp.append(metric_list[6])
            s_tn.append(metric_list[7])
            s_fp.append(metric_list[8])
            s_fn.append(metric_list[9])
            s_npv.append(metric_list[10])
            s_lrp.append(metric_list[11])
            s_lrm.append(metric_list[12])
            
            aucs.append(roc_auc)
            praucs.append(prec_rec_auc)
            aveprecs.append(ave_prec)
            
        results = {'Balanced Accuracy': mean(s_bac), 'Accuracy': mean(s_ac), 
                   'F1_Score': mean(s_f1), 'Sensitivity (Recall)': mean(s_re), 
                   'Specificity': mean(s_sp),'Precision (PPV)': mean(s_pr), 
                   'TP': mean(s_tp), 'TN': mean(s_tn), 'FP': mean(s_fp), 
                   'FN': mean(s_fn), 'NPV': mean(s_npv), 'LR+': mean(s_lrp), 
                   'LR-': mean(s_lrm), 'ROC_AUC': mean(aucs),'PRC_AUC': mean(praucs), 
                   'PRC_APS': mean(aveprecs)}
        print(results)

In [6]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        print_results(algorithm, full_path)

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'Balanced Accuracy': 0.6227824463118581, 'Accuracy': 0.6424242424242425, 'F1_Score': 0.5345616443177419, 'Sensitivity (Recall)': 0.5396825396825397, 'Specificity': 0.7058823529411765, 'Precision (PPV)': 0.5304692891649413, 'TP': 11, 'TN': 24, 'FP': 10, 'FN': 9, 'NPV': 0.7135854341736695, 'LR+': 1.8518197851531188, 'LR-': 0.6538762364849321, 'ROC_AUC': 0.638188608776844, 'PRC_AUC': 0.5622552868527515, 'PRC_APS': 0.4961754551912484}
Algorithm: Logistic Regression
{'Balanced Accuracy': 0.7100840336134454, 'Accuracy': 0.7090909090909091, 'F1_Score': 0.6511283376399656, 'Sensitivity (Recall)': 0.7142857142857143, 'Specificity': 0.7058823529411765, 'Precision (PPV)': 0.6019679554162313, 'TP': 15, 'TN': 24, 'FP': 10, 'FN': 6, 'NPV': 0.802742060806577, 'LR+': 2.4645502645502644, 'LR-': 0.40083065083065084, 'ROC_AUC': 0.7824463118580766, 'PRC_AUC': 0.6765544222430395, 'PRC

## From Pickle: Extract list of true and false positive rates for constructing ROC

In [7]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path+ '/' + each
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        # Define evaluation stats variable lists
        tprs = [] # true postitive rates
        mean_fpr = np.linspace(0, 1, 100) # used to plot all CVs in single ROC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            # Load pickled metric file for given algorithm and cv =
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fpr = results[1]
            tpr = results[2]

            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0

        results = {'tprs': np.mean(tprs, axis=0)}
        
        print(results)
        #print('fprs: '+str(mean_fpr))

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'tprs': array([0.        , 0.02212984, 0.04425968, 0.06638952, 0.08851936,
       0.1106492 , 0.13277904, 0.15490888, 0.17703872, 0.19676355,
       0.20980783, 0.2228521 , 0.23644151, 0.25103033, 0.26561915,
       0.28020798, 0.2947968 , 0.30938562, 0.32397444, 0.33856326,
       0.35315208, 0.40870611, 0.41252205, 0.41633798, 0.42592593,
       0.43791887, 0.44991182, 0.46753247, 0.48661215, 0.50569184,
       0.51996152, 0.53358987, 0.54721821, 0.55555556, 0.56373256,
       0.57194164, 0.58066378, 0.58938592, 0.59830848, 0.60839346,
       0.61847844, 0.62856341, 0.63864839, 0.64873337, 0.65705467,
       0.66168831, 0.66632195, 0.67095559, 0.67558923, 0.68022286,
       0.68808188, 0.69916627, 0.71025065, 0.72133504, 0.73241943,
       0.74350382, 0.75458821, 0.7656726 , 0.77675699, 0.78287867,
       0.78747337, 0.79206807, 0.79666277, 0.80125747, 0.8058521

## From Pickle: Extract list of precision and recall values for constructing PRC

In [8]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        # Define evaluation stats variable lists
        precs = [] # true postitive rates
        mean_recall = np.linspace(0, 1, 100) # used to plot all CVs in single PRC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            prec = results[4]
            recall = results[5]

            precs.append(interp(mean_recall, recall, prec))

        results = {'precs': np.mean(precs, axis=0)}

        print(results)
        #print('recall: '+str(mean_recall))

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'precs': array([1.        , 0.98428732, 0.96857464, 0.95286195, 0.93714927,
       0.92143659, 0.90572391, 0.89001122, 0.87429854, 0.85858586,
       0.84287318, 0.82716049, 0.81144781, 0.79573513, 0.78002245,
       0.76430976, 0.74859708, 0.7328844 , 0.71717172, 0.70145903,
       0.68574635, 0.67003367, 0.65432099, 0.63860831, 0.60005261,
       0.58505191, 0.57005121, 0.55505051, 0.5400498 , 0.53386744,
       0.53121242, 0.5285574 , 0.52590238, 0.52324735, 0.52059233,
       0.51793731, 0.51528229, 0.51262726, 0.51170526, 0.51511579,
       0.51852633, 0.52193686, 0.5253474 , 0.5297276 , 0.53483505,
       0.53994249, 0.54504994, 0.55015739, 0.53797119, 0.53561151,
       0.53325183, 0.53089214, 0.51994802, 0.51860834, 0.51726866,
       0.51592898, 0.51458931, 0.51365547, 0.51326276, 0.51287006,
       0.51247735, 0.51208464, 0.48734635, 0.48330461, 0.479262

## From Pickle: Extract Average Model Feature Importance Estimates (Over CVs)

In [9]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Dataset: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    original_headers = pd.read_csv(full_path + "/exploratory/ProcessedFeatureNames.csv", sep=',').columns.values.tolist() # Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        # Define evaluation stats variable lists
        FI_ave = [0] * len(original_headers)  # used to save average FI scores over all cvs. (all original features in dataset prior to feature selection included)
        
        for cv_count in range(0, cv_partitions): # loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fi = results[8]
            
            # Format feature importance scores as list (takes into account that all features are not in each CV partition)
            tempList = []
            j = 0
            headers = pd.read_csv(full_path + '/CVDatasets/' + each + '_CV_' + str(cv_count) + '_Test.csv').columns.values.tolist()
            if instance_label != None: 
                headers.remove(instance_label)
            headers.remove(outcome_label)
            for feature in original_headers:
                if feature in headers:  # Check if current feature from original dataset was in the partition
                    # Deal with features not being in original order (find index of current feature list.index()
                    f_index = headers.index(feature)
                    FI_ave[j] += fi[f_index]
                j += 1
            
        #Turn FI sums into averages
        for i in range(0, len(FI_ave)):
            FI_ave[i] = FI_ave[i] / float(cv_partitions)

        fi_dict = {}
        for key in original_headers:
            for value in FI_ave:
                fi_dict[key] = value
                FI_ave.remove(value)
                break  
                
        print(fi_dict)

Dataset: ['hcc_data', 'hcc_data_custom']
---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'Gender': 0.0, 'Symptoms': 0.0, 'Alcohol': 0.0, 'Hepatitis B Surface Antigen': 0.0, 'Hepatitis B e Antigen': 0.0, 'Hepatitis B Core Antibody': 0.0, 'Hepatitis C Virus Antibody': 0.0, 'Cirrhosis': 0.0, 'Endemic Countries': 0.0, 'Smoking': 0.0, 'Diabetes': 0.0, 'Obesity': 0.0, 'Hemochromatosis': 0.0, 'Arterial Hypertension': 0.0, 'Chronic Renal Insufficiency': 0.0, 'Human Immunodeficiency Virus': 0.0, 'Nonalcoholic Steatohepatitis': 0.0, 'Esophageal Varices': 0.0, 'Splenomegaly': 0.0, 'Portal Hypertension': 0.0, 'Portal Vein Thrombosis': 0.0, 'Liver Metastasis': 0.035235760971055095, 'Radiological Hallmark': 0.0, 'Age at diagnosis': 0.024112978524743225, 'Grams of Alcohol per day': 0.0, 'Packs of cigarets per year': 0.01684173669467786, 'Performance Status*': 0.053046218487394964, 'Encephalopathy degree*': 0.0, 'Ascites degre

## Extract and Output Testing Data Prediction Probabilities 

In [10]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")

    full_path = experiment_path + '/' + each
    
    # Make folder in experiment folder/datafolder to store all prediction probabilities per algorithm/CV combination
    if not os.path.exists(full_path + '/model_evaluation/prediction_probas'):
        os.mkdir(full_path + '/model_evaluation/prediction_probas')
        
    original_headers = pd.read_csv(full_path + "/exploratory/OriginalFeatureNames.csv", sep=',').columns.values.tolist() #Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))

        for cv_count in range(0,cv_partitions): #loop through cv's
            print("CV: "+str(cv_count))
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Load associated testing dataset
            test_data = pd.read_csv(full_path + '/CVDatasets/'+each+'_CV_' + str(cv_count) + '_Test.csv')
            probas_summary = test_data[[outcome_label,instance_label]]

            #Separate pickled results
            probas_ = results[9]
            print(probas_[:,1])
            probas_summary['1_prob'] = probas_[:,1]
            file_name = full_path + '/model_evaluation/prediction_probas/' + algorithm + '_CV_'+str(cv_count) + '_class1_probas.csv'
            probas_summary.to_csv(file_name, index=False)

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
CV: 0
[0.40963855 0.         0.         0.65384615 0.73913043 0.
 0.65384615 0.65384615 1.         1.         0.73913043 0.0974212
 0.         0.         0.0974212  0.40963855 0.         0.40963855
 1.         0.40963855 1.         0.0974212  0.73913043 1.
 0.         0.         0.         0.40963855 0.40963855 0.28813559
 0.         0.0974212  0.         0.86624204 0.73913043 0.28813559
 0.         0.         1.         0.73913043 0.65384615 0.40963855
 0.         0.73913043 1.         0.         0.65384615 0.0974212
 0.0974212  0.         1.         0.         0.40963855 0.65384615
 0.        ]
CV: 1
[0.61818182 0.03472932 0.03472932 0.44736842 1.         0.44736842
 1.         0.28813559 0.         0.51908397 0.03472932 0.51908397
 0.03472932 1.         0.         0.03472932 0.03472932 0.03472932
 0.03472932 1.         0.03472932 0.28813559 1.         0.51908397